In [10]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import txtai
import os
import time
import pandas as pd
from scipy.spatial.distance import cosine
import pickle
from tqdm import tqdm 

In [11]:
dataset = pd.read_excel('/Users/jd/trauma_llm/trauma_data/embedding_data_reorganized.xlsx')

total_response = 2

res = dataset['Value 1']
for ii in range(total_response-1):
    res += dataset['Value '+str(ii+2)]
res /= total_response

dataset['score'] = res

In [12]:
dataset.head()

,Response 1,Response 2,Value 1,Value 2,score
0,Obtain a trauma series of xrays followed by a ...,Perform primary trauma survey with attention t...,5,3,2.5
1,"Primary, secondary survey, X-ray of RLE to eva...",splint the leg and transfer them to a level 1 ...,8,4,4.0
2,Immediate transfer to trauma center. Consider ...,Stabilize and transfer,9,4,4.5
3,"1. Check VS and perform a primary survey, obta...","Primary Survey, follow ATLS, CXR/PXR w/ RLE im...",6,4,3.0
4,Transfer emergently to a Level 1 or 2 trauma c...,"I would obtain an anticoagulation history, ass...",9,4,4.5


In [13]:
'''# Load dataset
dataset = load_dataset("sentence-transformers/stsb", split="train")
#dataset = load_dataset("tabilab/biosses", split="train")
print(f"Loaded {len(dataset)} examples. Sample:")
print(dataset[0], "\n")'''

'# Load dataset\ndataset = load_dataset("sentence-transformers/stsb", split="train")\n#dataset = load_dataset("tabilab/biosses", split="train")\nprint(f"Loaded {len(dataset)} examples. Sample:")\nprint(dataset[0], "\n")'

In [14]:
# Generate embeddings for the first few sentence pairs (do not run without need, save results, costs money)
model_name = 'pubmedbert-base-embeddings'#'nomic-embed-text-v1.5'
dims = [768]#[64, 256, 512, 768]

for dim in dims:
    print('Doing dimension ', dim)
    file_to_save = 'trauma/'+model_name+'_'+str(dim)+'.pickle'
    embeddings = pd.DataFrame()
    sentence1 = []
    sentence2 = []
    embedding1 = []
    embedding2 = []
    cosine_similarity = []
    human_score = []

    model = SentenceTransformer("neuml/"+model_name)
    print('Extracting embedding and measuring similarity ...')
    time.sleep(1)
    for idx in tqdm(range(len(dataset))):
        first_sentence = dataset["Response 1"][idx]
        emb1 = response = list(model.encode(first_sentence))
        sentence1.append(
            first_sentence
        )
        embedding1.append(
            emb1
        )
        #############################
        second_sentence = dataset["Response 2"][idx]
        emb2 = list(model.encode(second_sentence))
        sentence2.append(
            second_sentence
        )
        embedding2.append(
            emb2
        )
        
        ############################
        cosine_similarity.append(
            1 - cosine(emb1, emb2)
        )
        human_score.append(
            dataset["score"][idx]/5.0#/4.0 #comment out this depending on the data
        )
        
        time.sleep(1)  # respect rate limits!

    embeddings['Sentence 1'] = sentence1
    embeddings['Sentence 2'] = sentence2
    embeddings['Embedding of Sentence 1'] = embedding1
    embeddings['Embedding of Sentence 2'] = embedding2
    embeddings['Cosine similarity'] = cosine_similarity
    embeddings['Human score'] = human_score

    with open(file_to_save, 'wb') as f:
        pickle.dump(embeddings, f)

Doing dimension  768
Extracting embedding and measuring similarity ...


100%|██████████| 234/234 [04:01<00:00,  1.03s/it]


In [15]:
embeddings.head()

,Sentence 1,Sentence 2,Embedding of Sentence 1,Embedding of Sentence 2,Cosine similarity,Human score
0,Obtain a trauma series of xrays followed by a ...,Perform primary trauma survey with attention t...,"[-0.3306599, 0.8632637, -0.24484287, 0.0431079...","[-0.44366592, 0.15227936, 0.27948096, -0.55667...",0.644739,0.5
1,"Primary, secondary survey, X-ray of RLE to eva...",splint the leg and transfer them to a level 1 ...,"[0.023162665, 0.70736414, 0.10501392, -0.91014...","[-0.120074004, 0.644543, 0.34221494, -0.835608...",0.510914,0.8
2,Immediate transfer to trauma center. Consider ...,Stabilize and transfer,"[-0.021241408, 0.32234025, -0.37472755, -0.867...","[-0.806114, 1.079065, -0.33295536, -0.93139726...",0.358710,0.9
3,"1. Check VS and perform a primary survey, obta...","Primary Survey, follow ATLS, CXR/PXR w/ RLE im...","[-0.2869435, -0.17030887, 0.1301573, -0.676078...","[-0.09868311, 0.23517676, 0.34942934, -0.80131...",0.635172,0.6
4,Transfer emergently to a Level 1 or 2 trauma c...,"I would obtain an anticoagulation history, ass...","[0.04616001, -0.061915644, 0.28549746, -1.4858...","[0.23398177, 0.51667047, -0.026331807, -1.0569...",0.463684,0.9
